# Fichier pour générer les soumissions et prédictions

### Se sert des fichiers .sav du model et de scaler

In [1]:
# librairies 
#----------------------------------------------
#----- Calculs 
import pandas as pd
import numpy as np
from numpy import mean, std
from math import sqrt, log
#save model 
import pickle 

### Récupération du test set d'évaluation Kaggle

In [4]:
# Lecture du fichier des données tests
#----------------------------------------------
PATH = '.' #modifier si besoin le chemin du fichier

# récupération du fichier contentant l'ensemble des données issues des requêtes
# concaténé avec les hotêls et nouvelles data
eval_set_kaggle_detailed = pd.read_csv(PATH + '//test_set_cleaned_addedFeatures.csv')

eval_set_kaggle = pd.read_csv(PATH + '//test_set.csv')
idx = eval_set_kaggle ["index"]

In [5]:
# Assigner le bon type aux variables
#----------------------------------------------
#--- Convert to integer: 
int_list = ["date","avatar_id","hotel_id","stock","request_number","request_nb", "pib", "price_m2", "nb_tourists", "nb_hab_km2"] 
eval_set_kaggle_detailed[int_list] = eval_set_kaggle_detailed[int_list].astype(int) 
#--- Convert to categorical: 
eval_set_kaggle_detailed["city"] = pd.Categorical(eval_set_kaggle_detailed["city"],ordered=False)
eval_set_kaggle_detailed["language"] = pd.Categorical(eval_set_kaggle_detailed["language"],ordered=False)
eval_set_kaggle_detailed["mobile"] = pd.Categorical(eval_set_kaggle_detailed["mobile"],ordered=False)
eval_set_kaggle_detailed["parking"] = pd.Categorical(eval_set_kaggle_detailed["parking"],ordered=False)
eval_set_kaggle_detailed["pool"] = pd.Categorical(eval_set_kaggle_detailed["pool"],ordered=False)
eval_set_kaggle_detailed["children_policy"] = pd.Categorical(eval_set_kaggle_detailed["children_policy"],ordered=False)
eval_set_kaggle_detailed["group"] = pd.Categorical(eval_set_kaggle_detailed["group"],ordered=False)
eval_set_kaggle_detailed["brand"] = pd.Categorical(eval_set_kaggle_detailed["brand"],ordered=False)

'# Assigner le bon type aux variables\n#----------------------------------------------\n#--- Convert to integer: \nint_list = ["date","avatar_id","hotel_id","stock","request_number"] \neval_set_kaggle_detailed[int_list] = eval_set_kaggle_detailed[int_list].astype(int) \n#--- Convert to categorical: \neval_set_kaggle_detailed["city"] = pd.Categorical(eval_set_kaggle_detailed["city"],ordered=False)\neval_set_kaggle_detailed["language"] = pd.Categorical(eval_set_kaggle_detailed["language"],ordered=False)\neval_set_kaggle_detailed["mobile"] = pd.Categorical(eval_set_kaggle_detailed["mobile"],ordered=False)\neval_set_kaggle_detailed["parking"] = pd.Categorical(eval_set_kaggle_detailed["parking"],ordered=False)\neval_set_kaggle_detailed["pool"] = pd.Categorical(eval_set_kaggle_detailed["pool"],ordered=False)\neval_set_kaggle_detailed["children_policy"] = pd.Categorical(eval_set_kaggle_detailed["children_policy"],ordered=False)\neval_set_kaggle_detailed["group"] = pd.Categorical(eval_set_kagg

In [ ]:
encoder_toutes_les_variables_pareil=False
if not encoder_toutes_les_variables_pareil:
####################target encoding ############################# manuellement (à voir si on veut faire la médiane un jour )
#1) recréer le tableau total avec les prix (temp=X_train concat avec Y_train)
    # Compute target mean : #groupby : permet de calculer la moyenne de prix par hotel id 
    #mean_price_by_hotel_id = temp.groupby("hotel_id")["price"].mean() #dictionnaire
    frequency_by_hotel_pool = pickle.load(open( "./dictionnary/pool", 'rb')) #on peut ensuite le recharger 
    frequency_by_hotel_child = pickle.load(open( "./dictionnary/child", 'rb')) #dictionnaire
    frequency_by_hotel_parking = pickle.load(open( "./dictionnary/parking", 'rb')) #dictionnaire
    frequency_by_hotel_mobile = pickle.load(open( "./dictionnary/mobile", 'rb'))  #dictionnaire

    #groupby : permet de calculer la variance de prix par hotel id 
    #std_price_by_hotel_id = temp.groupby("hotel_id")["price"].std()
    #groupby : permet de calculer la médiane de prix par hotel id 
    #median_price_by_hotel_id = temp.groupby("hotel_id")["price"].median()

    #appliquer ces nouvelles transformations à notre modèle
    #X_train["std_price_by_hotel_id"]=X_train['hotel_id'].map(std_price_by_hotel_id)
    #X_train['mean_price_by_hotel_id'] = X_train['hotel_id'].map(mean_price_by_hotel_id)
    total_reindex['pool'] = total_reindex['pool'].apply(lambda x : frequency_by_hotel_pool[x])
    total_reindex['children_policy'] = total_reindex['children_policy'].apply(lambda x : frequency_by_hotel_child[x])
    total_reindex['mobile'] = total_reindex['mobile'].apply(lambda x : frequency_by_hotel_mobile[x])
    total_reindex['parking'] = total_reindex['parking'].apply(lambda x : frequency_by_hotel_parking[x])

    #X_test['std_price_by_hotel_id']= X_test['hotel_id'].map(std_price_by_hotel_id)
    #X_test['mean_price_by_hotel_id'] = X_test['hotel_id'].map(mean_price_by_hotel_id)
    print("encodage fréquentiel de mobile, children_policy, parking et pool faits")
    
    #--- Convert to categorical: 
    float_list = ["parking", "pool", "mobile", "children_policy"] 
    total_reindex[float_list] = total_reindex[float_list].astype(float) 
    print(total_reindex)
    target_encoding=pickle.load(open( "./dictionnary/target_encoding", 'rb')) 
    total_reindex_encoding = target_encoding.transform(total_reindex)


### Tranformation de la variable quantitative 'stock' comme pour l'apprentissage

In [6]:
eval_set_kaggle_detailed["stock"]=eval_set_kaggle_detailed["stock"].map(lambda x: log(x+1))

In [7]:
# Variables explicatives
#-----------------------------------------------
# Liste des variables categorical: 
cat_list1 = ["city", "language", "brand","mobile","parking","pool","children_policy"] #avec brand
cat_list2 = ["city", "language", "group","mobile","parking","pool","children_policy"] #avec group
cat_list3 = ["city", "language", "brand", "group","mobile","parking","pool","children_policy", "hotel_id","request_number"] #avec brand & group

# Liste des variables quantitatives: 
quant_list1 = ["hotel_id", "stock","request_nb", "date", "pib", "price_m2", "nb_tourists", "nb_hab_km2"]  #on a enlevé avatar_id et price (=variable à expliquer)

# Création de la dataframe train
eval_set_kaggle_detailed_pricingDum  = pd.get_dummies(eval_set_kaggle_detailed[cat_list3],drop_first = True)
eval_set_kaggle_detailed_pricingQual  = eval_set_kaggle_detailed[cat_list3]
eval_set_kaggle_detailed_pricingQuant = eval_set_kaggle_detailed[quant_list1]

dfC_eval_set_kaggle = pd.concat([eval_set_kaggle_detailed_pricingQual ,eval_set_kaggle_detailed_pricingQuant],axis=1)
dfC_eval_set_kaggle.columns

Index(['city', 'language', 'brand', 'group', 'mobile', 'parking', 'pool',
       'children_policy', 'hotel_id', 'request_number', 'stock', 'date'],
      dtype='object')

In [8]:
eval_set_kaggle_detailed

,request_number,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
0,1,vilnius,21,romanian,0,1,161,3.850148,Boss Western,J.Halliday Inn,1,0,0
1,1,vilnius,21,romanian,0,1,187,3.496508,Accar Hotels,Marcure,1,1,0
2,1,vilnius,21,romanian,0,1,279,2.564949,Independant,Independant,1,0,0
3,1,vilnius,21,romanian,0,1,395,2.397895,Accar Hotels,Ibas,0,0,0
4,1,vilnius,21,romanian,0,1,488,3.761200,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6639,843,rome,5,irish,0,794,987,0.693147,Accar Hotels,Ibas,1,0,0
6640,844,vienna,1,irish,1,794,26,0.693147,Accar Hotels,Marcure,1,0,0
6641,844,vienna,1,irish,1,794,263,0.000000,Boss Western,Boss Western,1,0,0
6642,844,vienna,1,irish,1,794,456,0.000000,Yin Yang,Ardisson,1,0,0


### Obtention du pipeline

Les noms sont à adapter : charger les modèles et standard scaler correspondant(date)

In [11]:
model_folder = "/models_sav"
loaded_model = pickle.load(open(PATH + model_folder +"/2000_XGboost_target_encoding.sav", 'rb'))

/usr/local/insa/anaconda/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Prédiction et enregistrement csv

In [12]:
pred_eval = loaded_model.predict(dfC_eval_set_kaggle)
predict =pd.DataFrame(list(zip(idx, pred_eval**2)), columns=['index', 'price']) #### Mettre la prediction au carré car tranfor sqrt dans la transformation des données
predict.to_csv('submission.csv', index=False)
print(predict)

      index       price
0         0  114.642639
1         1  144.299026
2         2   57.844692
3         3   57.087265
4         4  190.192581
...     ...         ...
6639   6639   80.501289
6640   6640  191.581085
6641   6641  136.345383
6642   6642  180.029724
6643   6643  138.280319

[6644 rows x 2 columns]


In [65]:
loaded_model

RandomForestRegressor(n_estimators=500, oob_score=True)